In [1]:
!pip install -q gradio sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

In [2]:
# Gradio-based Resume Screening (Google Colab Compatible) with RAG-style LLM Generation + Evaluation
import gradio as gr
import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import faiss
import tempfile
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score
from transformers import pipeline
from functools import lru_cache

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')
llm = pipeline("text2text-generation", model="google/flan-t5-large", max_new_tokens=150, do_sample=True, temperature=0.9)

# Globals for caching FAISS index and data
df = None
resume_embeddings = None
index = None

# Clean resume text
def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', text.replace('\n', ' ').replace('\r', ' ')).strip()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [4]:
# Load and process CSV file
def load_csv(file):
    global df, resume_embeddings, index
    df = pd.read_csv(file.name)
    df['Cleaned_Resume'] = df['Resume'].apply(clean_text)
    resume_embeddings = model.encode(df['Cleaned_Resume'].tolist(), show_progress_bar=True)
    dimension = resume_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(resume_embeddings))
    return "✅ Resumes loaded and indexed. Paste a job description below."


In [5]:
# Cached generation function
@lru_cache(maxsize=128)
def generate_feedback(prompt):
    return llm(prompt)[0]['generated_text'].strip()

# Perform job matching with LLM generation and evaluation
def find_candidates(job_desc):
    if df is None:
        return "Please upload a dataset first.", None

    query_embedding = model.encode([job_desc])[0]
    _, all_indices = index.search(np.array([query_embedding]), k=len(df))
    all_indices = all_indices[0]

    required_skills = set(re.findall(r'\b\w+\b', job_desc.lower()))
    scoring_data = []

    # Step 1: Calculate match scores for all resumes
    for idx in all_indices:
        resume_text = df.iloc[idx]['Resume']
        candidate_skills = set(re.findall(r'\b\w+\b', resume_text.lower()))
        matching = required_skills & candidate_skills
        missing = required_skills - candidate_skills
        match_score = round(len(matching) / len(required_skills) * 100, 2) if required_skills else 0
        summary = resume_text[:300] + "..."

        scoring_data.append({
            "Candidate Index": idx,
            "Category": df.iloc[idx]['Category'],
            "Match Score (%)": match_score,
            "Matching Skills": ", ".join(sorted(matching)),
            "Missing Skills": ", ".join(sorted(missing)),
            "Summary": summary
        })

    # Step 2: Take top 5 only
    top_candidates = sorted(scoring_data, key=lambda x: -x["Match Score (%)"])[0:5]

    # Step 3: Add LLM feedback only for top 5
    for candidate in top_candidates:
        prompt = (
            f"You are an AI hiring assistant. Here is candidate #{candidate['Candidate Index']}.\n"
            f"Job Description: {job_desc}\n"
            f"Resume Excerpt: {candidate['Summary']}\n"
            f"Matching Skills: {candidate['Matching Skills']}\n"
            f"Missing Skills: {candidate['Missing Skills']}\n"
            f"Write a short, professional summary of this candidate's suitability for the role."
        )
        candidate["AI Assessment Feedback"] = generate_feedback(prompt)

    # Step 4: Evaluation Metrics (based on keyword overlap)
    retrieved_idxs = [c["Candidate Index"] for c in top_candidates]
    relevant_idxs = [i for i in range(len(df)) if df.iloc[i]['Category'].lower() in job_desc.lower()]
    relevant_set = set(relevant_idxs)
    retrieved_set = set(retrieved_idxs)

    recall_at_5 = len(retrieved_set & relevant_set) / len(relevant_set) if relevant_set else 0
    precision_at_5 = len(retrieved_set & relevant_set) / 5

    # Step 5: Prepare markdown output
    results = [f"**Recall@5:** {recall_at_5:.2f} | **Precision@5:** {precision_at_5:.2f}\n---"]
    for row in top_candidates:
        results.append(f"**Category:** {row['Category']}\n\n**Match Score:** {row['Match Score (%)']}%\n\n**Matching Skills:** {row['Matching Skills']}\n\n**Missing Skills:** {row['Missing Skills']}\n\n**Summary:** {row['Summary']}\n\n**AI Assessment Feedback:** {row['AI Assessment Feedback']}\n\n---")

    # Step 6: Export to Excel
    export_df = pd.DataFrame(top_candidates)
    export_df["Recall@5"] = recall_at_5
    export_df["Precision@5"] = precision_at_5

    with tempfile.NamedTemporaryFile(delete=False, suffix=".xlsx") as tmp_file:
        export_df.to_excel(tmp_file.name, index=False, sheet_name="Screening Results")
        excel_path = tmp_file.name

    return "\n".join(results), excel_path

In [6]:
# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 📄 Resume Screening App + RAG Evaluation")
    with gr.Row():
        file_input = gr.File(label="Upload Resume Dataset (CSV)")
        upload_output = gr.Textbox(label="Status")
    file_input.change(fn=load_csv, inputs=file_input, outputs=upload_output)

    job_desc_input = gr.Textbox(lines=5, placeholder="Paste job description here...", label="Job Description")
    with gr.Row():
        match_btn = gr.Button("Find Candidates")
        download_output = gr.File(label="📥 Download Excel")
    output_box = gr.Markdown()

    match_btn.click(fn=find_candidates, inputs=job_desc_input, outputs=[output_box, download_output])

demo.launch()



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://85334a03a71fa28af3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
